In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt

In [2]:
with open('input.txt', 'r') as input:
    n = int(input.readline())
    complexity = np.array(list(map(int, input.readline().split())))
    time = np.array(list(map(float, input.readline().split())))
    m = int(input.readline())
    coeffs = np.array([list(map(float, line.split())) for line in input])
    input.close()

In [3]:
coeffs, n, m, complexity, time

(array([[0.54, 1.17, 1.52, 2.18],
        [0.31, 0.66, 1.13, 1.76],
        [0.48, 1.09, 1.78, 2.39],
        [0.61, 1.09, 1.5 , 2.04],
        [0.66, 1.02, 1.42, 1.79],
        [0.44, 0.77, 1.16, 1.47],
        [0.59, 1.23, 1.59, 2.13],
        [0.49, 1.08, 1.67, 2.35],
        [0.55, 1.  , 1.3 , 1.67],
        [0.37, 0.8 , 1.44, 2.01]]),
 1000,
 10,
 array([2, 3, 1, 2, 4, 4, 2, 2, 2, 4, 3, 3, 4, 4, 1, 3, 2, 4, 4, 1, 3, 1,
        2, 2, 3, 2, 3, 2, 4, 4, 4, 3, 2, 3, 4, 4, 3, 1, 3, 4, 1, 1, 2, 2,
        3, 2, 1, 3, 4, 3, 2, 2, 4, 2, 2, 1, 2, 3, 2, 3, 2, 4, 3, 2, 3, 1,
        3, 2, 1, 4, 2, 4, 4, 4, 3, 2, 2, 1, 4, 1, 2, 4, 2, 1, 4, 1, 2, 2,
        4, 3, 2, 3, 3, 1, 1, 3, 2, 1, 4, 4, 4, 4, 1, 2, 2, 1, 4, 1, 3, 2,
        3, 1, 4, 4, 3, 3, 1, 1, 4, 1, 4, 1, 1, 2, 2, 4, 1, 3, 3, 3, 3, 4,
        1, 4, 2, 4, 2, 1, 1, 2, 1, 3, 2, 4, 3, 4, 2, 4, 3, 1, 3, 4, 2, 4,
        3, 2, 3, 2, 3, 4, 1, 4, 3, 4, 2, 2, 3, 3, 4, 4, 2, 4, 2, 4, 2, 3,
        1, 4, 1, 1, 2, 4, 2, 2, 4, 4, 3, 3, 4, 2, 2, 2

In [206]:
class GeneticAlgorithm:

    def __init__(self,
                 n: int,
                 complexity: np.ndarray,
                 time: np.ndarray,
                 m: int,
                 coeffs: np.ndarray,
                 size_population: int,
                 size_selection: int,
                 p_mutation_ind: float,
                 p_mutation_gen: float
        ):
        self.n = n                                      # Количество задач
        self.complexity = complexity                    # Категории сложности задач (от 1 до 4)
        self.time = time                                # Оценочное время для задач
        self.m = m                                      # Количество разработчиков
        self.coeffs = coeffs                            # Коэффициенты разработчика (на каждую категорию сложности)

        self.size_population = size_population          # Количество особей в популяции
        self.size_selection = size_selection            # Выживающие особи во время селекции
        self.p_mutation_ind = p_mutation_ind            # Вероятность мутации детей
        self.p_mutation_gen = p_mutation_gen            # Вероятность мутации генов

        self._score = 0                                 # Показатель качества

        self.rng = np.random.default_rng()              # Генератор случайных чисел
        
        self.population = self.rng.integers(1, m + 1, size=(self.size_population, self.n))

    def fitness(self) -> np.ndarray:
        sum_time_for_dev = np.zeros((self.size_population, self.m)) # Массив с временем каждого разработчика для всех задач

        for indIdx, individ in enumerate(self.population):
            individ_result = []
            task_number = 0
            for gen in individ:
                individ_result.append(self.coeffs[gen - 1, self.complexity[task_number] - 1] * self.time[task_number])
                task_number += 1
            for task_number in range(self.n):
                sum_time_for_dev[indIdx, individ[task_number] - 1] += individ_result[task_number]
        
        max_time = np.max(sum_time_for_dev, axis=1)

        self._score = self.calc_score(max_time)
        
        return max_time
    
    def calc_score(self, time) -> float:
        return 1e2 / np.min(time)

    def selection(self) -> None:
        self.selected = self.population[np.argsort(self.fitness())[:self.size_selection]]

    def crossover(self) -> None:
        new_count = self.size_population - self.size_selection
        parent1 = self.rng.integers(0, self.size_selection, size=new_count)
        parent2 = (self.rng.integers(1, self.size_selection, size=new_count) + parent1) % self.size_selection
        
        point1 = self.rng.integers(-1, self.n - 3, size=new_count)
        point2 = self.rng.integers(point1, self.n + 2, size=new_count)
        
        mask1 = np.arange(self.n)[None] >= point1[..., None]
        mask2 = np.arange(self.n)[None] <= point2[..., None]

        self.childs = np.where(
            mask1 == mask2,
            self.selected[parent1],
            self.selected[parent2]
        )        

    def mutation(self) -> None:
        mut_childs_mask = self.rng.choice(2, p=(1 - self.p_mutation_ind, self.p_mutation_ind), size=len(self.childs)) > 0
        mut_childs = self.rng.integers(1, m + 1, size=(mut_childs_mask.sum(), self.n))
        gen_childs_mask = self.rng.random(size=mut_childs.shape) <= self.p_mutation_gen
        self.childs[mut_childs_mask] = np.where(gen_childs_mask, mut_childs, self.childs[mut_childs_mask])

    def step(self) -> None:
        self.selection()
        self.crossover()
        self.mutation()
        self.population = np.concatenate([self.selected, self.childs], axis=0)

    @property
    def score(self):
        return self._score

In [255]:
gen = GeneticAlgorithm(n, complexity, time, m, coeffs, 420, 50, 0.67, 0.02)

In [257]:
for i in range(50):
    print(f"fitness: {np.round(np.min(gen.fitness()), 3)};\nscore: {np.round(gen.score, 5)};\niteration: {i}\n")
    gen.step()

fitness: 607.565;
score: 0.16459;
iteration: 0

fitness: 607.565;
score: 0.16459;
iteration: 1

fitness: 607.565;
score: 0.16459;
iteration: 2

fitness: 607.565;
score: 0.16459;
iteration: 3

fitness: 607.565;
score: 0.16459;
iteration: 4

fitness: 607.565;
score: 0.16459;
iteration: 5

fitness: 607.51;
score: 0.16461;
iteration: 6

fitness: 607.51;
score: 0.16461;
iteration: 7

fitness: 607.51;
score: 0.16461;
iteration: 8

fitness: 606.735;
score: 0.16482;
iteration: 9

fitness: 606.13;
score: 0.16498;
iteration: 10

fitness: 606.13;
score: 0.16498;
iteration: 11

fitness: 606.005;
score: 0.16502;
iteration: 12

fitness: 605.65;
score: 0.16511;
iteration: 13

fitness: 605.65;
score: 0.16511;
iteration: 14

fitness: 605.65;
score: 0.16511;
iteration: 15

fitness: 605.65;
score: 0.16511;
iteration: 16

fitness: 605.65;
score: 0.16511;
iteration: 17

fitness: 605.65;
score: 0.16511;
iteration: 18

fitness: 605.65;
score: 0.16511;
iteration: 19

fitness: 605.65;
score: 0.16511;
iteration

In [ ]:
# Запускал два раза, то есть 2 по 50 итераций

In [258]:
with open('output.txt', 'w') as output:
    for task in gen.population[np.argmin(gen.fitness())]:
        print(task, file=output, end=' ')

In [259]:
gen.population[np.argmin(gen.fitness())]

array([ 7,  6,  3,  6,  4,  3,  6,  7,  2, 10,  9,  9,  9,  1,  7,  8,  9,
        8,  3,  5,  4, 10, 10,  2,  5,  4,  8,  6,  2,  6,  8,  3, 10,  2,
        1,  1,  3,  3,  9,  3,  7,  9,  9,  1,  8,  4,  9,  2,  1,  6, 10,
        8,  3,  6,  1,  6,  8,  6, 10,  3,  5,  6,  1,  2,  4,  8,  3,  2,
        6,  9, 10,  6,  8,  2, 10,  9,  6,  9,  3,  2,  3,  3,  9,  7,  7,
        2,  6,  1,  6,  6,  7,  4, 10,  9,  4,  9,  3,  1, 10,  7,  9,  3,
        4,  2,  3,  6,  9, 10,  1,  8,  8,  7,  9,  1,  5,  9,  3, 10,  3,
        8,  4,  3,  3,  6,  6, 10,  9,  2,  9,  3,  9,  2,  1,  6, 10,  9,
        4,  9, 10,  7,  8,  8,  2,  1,  4,  9,  3,  2,  1,  5,  6,  6,  2,
        6,  8,  6,  2,  4, 10,  7,  1,  6,  2,  5,  3,  6,  2,  6,  2,  3,
        7,  8,  7,  1,  8,  2,  2,  7,  8,  5,  3,  4, 10,  7,  8,  5,  1,
        5,  2,  3,  2,  8,  5,  7,  5,  6,  2,  7,  4,  3, 10,  6,  7,  3,
        6,  7,  2,  2,  8,  9,  1,  6,  5,  8,  6,  9,  4,  8,  6,  3,  4,
        9,  7,  2,  8, 10